# SailGP Data Analyst Challenge

The aim is to test you python abilities. The challenge is to analyze the data provided and answer the questions below. You can use any library you want to help you with the analysis. The data is from the SailGP event in Auckland 2025. The data is in the 'DATA' folder.

There are various sources available.

The Boat Logs are in the 'Boat_Logs' folder. The data is in csv format and the columns are described in the 'Boat_Logs/Boat_Logs_Columns.csv' file.
The 'Course_Marks_2025-01-19.csv' file contains the mark positions and wind reading on the course for the whole day.

The Race_XML folder contains the xml files for each race that contains information on where the boundaries of the course are, the theoretical position of the marks and the target racecourse axis.

The 2025-01-19_man_summary.csv file contains the metrics from the manoeuvre summary for the day.
The 2025-01-19_straight_lines.csv file contains the metrics from the straight line summary for the day.

Both are derived from the boat logs.

The 2502 m8_APW_HSB2_HSRW.kph.csv file contains the polar data for the boats in that config.

## Requierements
- Chose at least 3 questions from the list below to answer.
- Python 3.8 or higher
- Notebook should be able to run without any errors from start to finish.
- Specify the libraries (imports) used in the notebook.
- Any comments to make the notebook self-explanatory and easy to follow would be appreciated.
- If you can't get to the end of a question, we would appreciate the code you have written so far and explain what you were trying to do.

## Further information:
- We usually use bokeh for visualizations. So any showcase of bokeh would be appreciated.
-

## Submitting the results.
It would be great if you could provide a jupyter notebook with the code and the results of the analysis. You can submit the results by sharing a link to a git repository.


### Imports and re-used functions
Free section to initialize the notebook with the necessary imports and functions that will be used in the notebook.



In [3]:
!git clone https://github.com/DrSailGP/SGP_Data_Challenge.git
!cd SGP_Data_Challenge

fatal: destination path 'SGP_Data_Challenge' already exists and is not an empty directory.


In [5]:
pip install pandas numpy bokeh matplotlib scipy math bs4

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#!jupyter notebook main_final.ipynb #change back to!jupyter notebook main.ipynb

[I 2025-02-08 21:38:06.803 ServerApp] Extension package aext_assistant took 0.6992s to import
[I 2025-02-08 21:38:06.814 ServerApp] ****************** ENVIRONMENT [Panels] Environment.PRODUCTION ******************
[I 2025-02-08 21:38:06.815 ServerApp] ****************** ENVIRONMENT [Panels] Environment.PRODUCTION ******************
[I 2025-02-08 21:38:06.906 ServerApp] --> Alembic Config: {'db_conn_url': 'sqlite:////Users/blindsay/anaconda_projects/db/project_filebrowser.db', 'db_path': '/Users/blindsay/anaconda_projects/db/project_filebrowser.db'}
[I 2025-02-08 21:38:06.906 ServerApp] Default dir: /Users/blindsay. Running chdir...
[I 2025-02-08 21:38:06.906 ServerApp] Checking for database file @ /Users/blindsay/anaconda_projects/db/project_filebrowser.db
[I 2025-02-08 21:38:06.906 ServerApp] Database file already exists
[I 2025-02-08 21:38:06.915 ServerApp] Migrations executed
[I 2025-02-08 21:38:06.915 ServerApp] Checking project configuration: /Users/blindsay. Running chdir...
[I 2

## Question 1: Write a Python function that can take a compass direction (ie. TWD or Heading) and calculate an accurate mean value across a downsampled frequency. Eg. If TWD is at 1Hz, give me a 10s average.

In [15]:
#DONE
import pandas as pd
import numpy as np 

def downsampled_mean (compass_dir, og_frequency, dwnsmpl_freq):
    
    for i in range (0, len(compass_dir), (og_frequency*dwnsmpl_freq)):
        ds = compass_dir[i:i + (og_frequency*dwnsmpl_freq)]
        ds_mean = np.mean(ds)

    return print(ds_mean)

In [19]:
#CODE TO CHECK THAT IT RUNS 
sample_df = pd.read_csv('SGP_Data_Challenge/Data/Boat_logs/data_BRA.csv')
compass_dir = sample_df['HEADING_deg']
#TWD is at 1Hz, give me a 10s average.
downsampled_mean(compass_dir, 1, 10)

#TWD is at 1Hz, give me a 100s average.
downsampled_mean(compass_dir, 1, 9)

109.18
118.21375


## Question 2: Given a course XML and a timeseries of boat Lat/Lon values, calculate a VMC column for the same timeseries.


In [137]:
#DONE 
from bs4 import BeautifulSoup
import math 

#get marks from XLM and return coordinates for legs
def find_mark(XML_file):
   
    with open(XML_file, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'xml')

    mark_coords = {} #find mark lat and long from XLM 
    for all_marks in soup.find_all("CompoundMark"):
        for mark in all_marks.findall("Mark"):
            ID_mark = mark.get('CompoundMarkID')
            if ID_mark:
                mark_coords[ID_mark] = (
                    float(mark.gat('TargetLat', 0)),
                    float(mark.get('TargetLng', 0))
                )
            
    course = []
    for corner in soup.find_all("Corner"):
        ID_mark = corner.get('CompoundMarkID')
        if ID_mark in mark_coords:
            course.append(mark_coords[ID_mark])

    legs = []
    for i in range(len(course) - 1):
        last_mark = course[i] #last mark sailed
        next_mark = course[i + 1] #next mark sailing to 
        legs.append((last_mark, next_mark))
    return legs

#calculate distance between coodinates using Spherical Law of Cosines
def distance(coord1, coord2):
    lat1, lon1 = map(math.radians, coord1)
    lat2, lon2 = map(math.radians, coord2)
    
   ##add explination for how SLC is being used *******
    diff_lon = lon2 - lon1
    dist = math.acos(math.sin(lat1) * math.sin(lat2) + math.cos(lat1) * math.cos(lat2) * math.cos(diff_lon)) * 6371 #earth radius in km

    return round(dist, 3)

#calculate heading in deg
def heading(coord1, coord2):
    lat1, lon1 = map(math.radians, coord1)
    lat2, lon2 = map(math.radians, coord2)

    diff_lon = lon2 - lon1
    x = math.sin(diff_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(diff_lon)

    heading = math.atan2(x,y)
    return round((math.degrees(heading) + 360) % 360,)

#caluclate VMC
def VMC(time_lat_lon, lat_lon_target):
    vmc = []

    for i in range(len(time_lat_lon) - 1):
        time1, lat1, lon1 = time_lat_lon[i]
        time2, lat2, lon2 = time_lat_lon[i +1]

    #time diff in sec
        diff_time = (time2 - time1).total_seconds()

    #boatstpeed (sog) in kts
        dist = distance((lat1, lon1), (lat2, lon2))
        sog = dist / (diff_time / 3600) #convert to kts
    
    #heading
        boat_heading = heading((lat1, lon1), (lat2, lon2))
        target_course = heading((lat2, lon2), lat_lon_target)

    #vmc = sog * cos(abs(heading - target_course)
        vmc_calc = round(sog * abs(math.cos(math.radians(abs(boat_heading - target_course)))), 2)
        vmc.append(vmc_calc)
        
    return vmc
    


In [ ]:
#CODE TO CHECK THAT IT RUNS 

file_path = "SGP_Data_Challenge/Data/Race_XMLs/25011905_03-13-55.xml"

from datetime import datetime
# time_lat_lon = [
#         (datetime(2025, 1, 19, 16, 6, 0), -36.8334, 174.759),
#         (datetime(2025, 1, 19, 16, 7, 0), -36.833, 174.759),
#         (datetime(2025, 1, 19, 16, 8, 0), -36.834, 174.769),
#         (datetime(2025, 1, 19, 16, 9, 0), -36.833, 174.768),
#         (datetime(2025, 1, 19, 16, 10, 0), -36.832, 174.767),
        
# ]

time_lat_lon = 
lat_lon_target= (-36.8297130, 174.7652700)

VMC(time_lat_lon,lat_lon_target)


## Question 3: Verify and comment on the boats calibration. If possible propose a post-calibrated set of wind numbers and a potential calibration table.


## Question 4: Given a timeseries of Lat/Lon positions and a course XML, in a Python notebook, calculate a Distance to Leader metric for each boat.

## Question 5: Given a course XML, along with a wind speed and direction and a polar, calculate the minimum number of tacks or gybes for each leg of the course and each gate mark on the leg.

## Question 6: Calculate a “tacked” set of variables depending on the tack of the boat, so that sailors don’t need to think about what tack they’re on when looking at measurements. And show the results in a visualisation.


## Question 7: Given a set of tacks (in CSV), and train a model to explain the key features of these tacks when optimizing for vmg. Show appropriate visualisations to explain your conclusions.

In [175]:
#Running a Random Forest Regressor 
#we have a mix of continuous and a few categorical variables
#random forestshould help with overfitting 

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from bokeh.palettes import Spectral11
from scipy.stats import zscore 

#load in maneuver dataset 
man_summary_19 = pd.read_csv('SGP_Data_Challenge/Data/2025-01-19_man_summary.csv')
tacks = man_summary_19[man_summary_19['type'] != 'gybe']

#seperate response y and explanatory x variables
y = tacks['theoretical_vmg']

X = tacks.drop(columns = ['theoretical_vmg', 'BOAT', 'HULL', 'DATETIME', 'TIME_LOCAL_unk',
                          'dashboard', 'htw_bsp', 'entry_tack', 'type', 'drop_time_P',
                          'drop_time_S', 'unstow_time_P', 'unstow_time_S', 'stow_time_P',
                          'stow_time_S', 'boards_up_time_S', 'boards_up_time_P', 
                          'max_lat_gforce', 'max_fwd_gforce', 'max_gforce', 'theoretical_distance',
                          'MD4_SEL_DB_unk', 'MD4_SEL_RUD_unk', 'WING_CONFIG_unk', 'leg', 'race'], axis=1)

#spit data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.02, random_state=42)

#initialize random forest regressor
model = RandomForestRegressor(n_estimators=200, random_state=42)

#train model
model.fit(X_train, y_train)

#evaluate the model
y_pred = model.predict(X_test)
MAE = mean_absolute_error(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
R2 = r2_score(y_test, y_pred)
print(f'Mean Absolute Error: {MAE}') #on avg predictions are off by 2.24 units from true value
print(f'Mean Squared Error: {MSE}') #MSE is only 10.5% of total range, so relativley small
print(f'Root Mean Squared Error: {RMSE}') #model error is about 2.9 units, decent accureacy 
print(f'R-squared: {R2}') #model explains 94% of variance in theoretical vmg, strong model 
#model has low error values and a high r squared, so it is good

#checking for outliers 
residuals = y_test - y_pred
residuals_z = np.abs(zscore(residuals))
outliers = residuals_z > 3 

print("num outliers:", np.sum(outliers))
print('outlier indicies:', np.where(outliers)[0])

# #regularizing model 
# param_grid = {
#     'n_estimators': [50,100,200],
#     'max_depth': [5, 10, 20, None],
#     "min_samples_split": [2, 5, 10],
#     "min_samples_leaf": [1,2,4],
#     "max_features": ["sqrt", "log2", None],
#     "bootstrap": [True, False]
# }

# rf = RandomForestRegressor(random_state = 42)

#grid search with 5-fold cross validation
# grid_search = GridSearchCV(rf, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
# grid_search.fit(X_train, y_train)

# best_params = grid_search.best_params_
# print(best_params)

# best_rf = RandomForestRegressor(**best_params, random_state = 42)
# best_rf.fit(X_train, y_train)

# y_pred = best_rf.predict(X_test)
# #model will benefit form shallower tree, bootstrapping, 200 estimators, all features 
# best_rf = RandomForestRegressor(
#     bootstrap= True,
#     max_depth =10, 
#     max_features=None, 
#     min_samples_leaf=1,
#     min_samples_split=2, 
#     n_estimators = 200,
#     random_state =42
# )

# best_rf.fit(X_train, y_train)
# y_pred = best_rf.predict(X_test)

# MAE2 = mean_absolute_error(y_test, y_pred)
# MSE2 = mean_squared_error(y_test, y_pred)
# RMSE2 = np.sqrt(MSE)
# R2_2 = r2_score(y_test, y_pred)
# print(f'Mean Absolute Error2: {MAE2}') #on avg predictions are off by 2.24 units from true value
# print(f'Mean Squared Error2: {MSE2}') #large prediction errors are contributing to overall error 
# print(f'Root Mean Squared Error2: {RMSE2}') #model error is about 2.9 units, decent accureacy 
# print(f'R-squared2: {R2_2}') #model explains 94% of variance in theoretical vmg, strong model 

#####show which variables contribute to vmg the most and which don't 
#show which of the explanatory variables are important to optimizing vmg
# important_vars = model.feature_importances_

# #create df with the feature names and their importances
# important_vars = important_vars.sort_values(by='Importance', ascending = False)
# threshold = .005
# significant_vars = important_vars[important_vars['Importance'] >= threshold]['Feature']

# #new df with only the important variables 
# X_important = X[significant_vars]

# #train new model just on these important Xs
# model2 = RandomForestRegressor(n_estimators=1000)
# model2.fit(X_important, y)



Mean Absolute Error: 2.3576061986098695
Mean Squared Error: 8.029454699028225
Root Mean Squared Error: 2.8336292451603873
R-squared: 0.9384132857024592
num outliers: 0
outlier indicies: []


## Question 8: Give insights on the racing on what made a team win or underperform in the race.